# Alpha–Phi Binary Explorer (v9c) — *Fine‑grid Log‑space Sweep*

* **Refined ‘a’ grid**: offsets every 0.0005 around `log10(α)` (±0.01).  
* **Slower contraction factors** `b` = 0.94…0.995.  
* **300 iterations** per combo (log map needs more steps when b ≈ 1).  
* Skips obvious outliers (`rel_err > 0.3`) for speed.  
* Stops at first **< 1 % hit** (prints if < 0.1 %).  
* Saves all sub‑1 % hits to `log_space_scan_v9c.csv`.

In [1]:
import mpmath as mp, csv, math
mp.mp.dps = 300
ALPHA = mp.mpf('7.2973525693e-3')
log_alpha = mp.log10(ALPHA)

def fib_word_bits(n):
    a,b='0','01'
    while len(b)<n:
        a,b=b,b+a
    return b[:n]

def fib_word_real(n):
    bits=fib_word_bits(n)
    return mp.fsum(int(b)*mp.mpf(2)**(-(k+1)) for k,b in enumerate(bits))


In [2]:
def iterate_log(y0, a, b, steps=300):
    for _ in range(steps):
        y0 = a - b*y0
    return y0

## Fine‑grid sweep

In [3]:
a_offsets = [i*0.0005 for i in range(-20,21)]  # ±0.01
a_vals = [log_alpha + mp.mpf(off) for off in a_offsets]
b_vals = [mp.mpf(str(b)) for b in (0.94,0.96,0.97,0.98,0.99,0.995)]
hits=0
with open('log_space_scan_v9c.csv','w',newline='') as f:
    w=csv.writer(f); w.writerow(['a','b','bits','rel_err'])
    for bits in (512,1024):
        y0 = mp.log10(fib_word_real(bits))
        for a in a_vals:
            for b in b_vals:
                y = iterate_log(y0,a,b,300)
                x = mp.power(10,y)
                rel = abs((x-ALPHA)/ALPHA)
                if rel > 0.3:
                    continue
                if rel < 0.01:
                    hits += 1
                    w.writerow([str(a),str(b),bits,str(rel)])
                    print('Hit <1%  ', mp.nstr(rel,8), 'params', (a,b,bits))
                    if rel < 0.001 or hits>=3:
                        raise SystemExit
print('No hit <1% in fine grid')


No hit <1% in fine grid
